In [3]:
# 本代码由可视化策略环境自动生成 2022年4月21日 22:52
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}
    equities = {e.symbol: e for e, p in context.portfolio.positions.items()}

    # 记录持仓中st的股票
    st_stock_list = []
    name_df = context.name_df
    name_today = name_df[name_df.date==data.current_dt.strftime('%Y-%m-%d')]
    for instrument in equities:
        try:
            name_instrument = name_today[name_today.instrument==instrument]['name'].values[0]
        except:
            continue
        # 如果股票名称中含有ST或退等字样则卖出
        if  '退' in name_instrument:
            context.order_target(context.symbol(instrument), 0)
            st_stock_list.append(instrument)
            cash_for_sell -= positions[instrument]
    if st_stock_list!=[]:
        print(data.current_dt.strftime('%Y-%m-%d'),'持仓出现退市股',st_stock_list,'进行卖出处理')
 
    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
            lambda x: x in equities)])))
        for instrument in instruments:
            # st/退市股票上面已经卖过了，此处跳过防止出现空单
            if instrument in st_stock_list:
                continue
            context.order_target(context.symbol(instrument),0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            try:
                name_instrument = name_today[name_today.instrument==instrument]['name'].values[0]
            except:
                continue
            if  '退' in name_instrument:
                pass
            else:
                context.order_value(context.symbol(instrument), cash)


# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    # 获取股票名称 用于过滤st和退市股
    context.name_df = DataSource('instruments_CN_STOCK_A').read()

# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0002, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 10


m1 = M.instruments.v2(
    start_date='2012-01-01',
    end_date='2018-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -10) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.03), all_quantile(label, 0.97))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""ta_atr_28_0/close_0*100

"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=350
)

m5 = M.chinaa_stock_filter.v1(
    input_data=m15.data,
    index_constituent_cond=['全部'],
    board_cond=['全部'],
    industry_cond=['全部'],
    st_cond=['正常'],
    output_left_data=False
)

m16 = M.derived_feature_extractor.v3(
    input_data=m5.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m13 = M.dropnan.v1(
    input_data=m7.data
)

m6 = M.stock_ranker_train.v5(
    training_ds=m13.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    m_lazy_run=False
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2018-01-01'),
    end_date=T.live_run_param('trading_date', '2019-04-04'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=350
)

m4 = M.chinaa_stock_filter.v1(
    input_data=m17.data,
    index_constituent_cond=['全部'],
    board_cond=['全部'],
    industry_cond=['全部'],
    st_cond=['正常'],
    output_left_data=False
)

m18 = M.derived_feature_extractor.v3(
    input_data=m4.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m14 = M.dropnan.v1(
    input_data=m18.data
)

m8 = M.stock_ranker_predict.v5(
    model=m6.model,
    data=m14.data,
    m_lazy_run=False
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m8.predictions,
    start_date='',
    end_date='',
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    initialize=m19_initialize_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2019-04-15 16:54:38.294005] INFO: bigquant: instruments.v2 开始运行..

[2019-04-15 16:54:38.392025] INFO: bigquant: 命中缓存

[2019-04-15 16:54:38.393764] INFO: bigquant: instruments.v2 运行完成[0.099754s].

[2019-04-15 16:54:38.396966] INFO: bigquant: advanced_auto_labeler.v2 开始运行..

[2019-04-15 16:54:43.404793] INFO: 自动标注(股票): 加载历史数据: 3654268 行

...

[2019-04-15 17:04:52.969025] INFO: algo: trading transform...

[2019-04-15 17:05:13.242624] INFO: Performance: Simulated 305 trading days out of 305.

[2019-04-15 17:05:13.244343] INFO: Performance: first open: 2018-01-02 09:30:00+00:00

[2019-04-15 17:05:13.245920] INFO: Performance: last close: 2019-04-04 15:00:00+00:00

[2019-04-15 17:05:16.797462] INFO: bigquant: backtest.v8 运行完成[75.661128s].

## 如果可以画出交易点可能会对改进算法有一定的帮助

In [4]:
def transaction_money(trade):
    '''
    输入trade
    输出前十个和后十个收益率的股票数据
    '''
    #交易详情
    df=trade.read_raw_perf()
    transactions=df.transactions.iloc[1:].values
    tras=None
    #链接成dataframe
    for i in transactions:
        if isinstance(tras,pd.DataFrame):        
            tras=pd.concat([tras,pd.DataFrame(i)])
        else:
            tras=pd.DataFrame(i)

    #改变sid形式
    tras.sid=tras.sid.apply(lambda x:x.symbol)
    #获取需要的数据
    tras=tras[['dt','sid','transaction_money']]
    
    #找到收益最大的10和最少的十个，这个是错误的 。
    transaction_money_sum=tras.groupby('sid').apply(lambda x:x.transaction_money.sum()).sort_values()
    top=transaction_money_sum.head(10)
    bottom=transaction_money_sum.tail(10)
    return top,bottom,tras

def get_bars(ins,start_date,end_date,signal):
    # Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
    def m5_run_bigquant_run(input_1, input_2, input_3):
        # 示例代码如下。在这里编写您的代码
        df = input_1.read()
        df=df.rename_axis({'open_0':'open','close_0':'close',"low_0":'low','high_0':'high'},axis='columns').set_index('date')
        ins=df.instrument.iloc[0]
        del df['instrument']    
        data_1 = DataSource.write_df(df)       
        return Outputs(data_1=data_1, data_2=ins, data_3=None)

    # 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
    def m5_post_run_bigquant_run(outputs):
        return outputs


    m1 = M.instruments.v2(
        start_date=start_date,
        end_date=end_date,
        market='CN_STOCK_A',
        instrument_list=[ins],
        max_count=0
    )

    m3 = M.input_features.v1(
        features="""
    # #号开始的表示注释，注释需单独一行
    # 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
    high_0
    close_0
    low_0
    open_0
    ta_sma_30_0   
    """+signal
        
    )

    m2 = M.general_feature_extractor.v7(
        instruments=m1.data,
        features=m3.data,
        start_date='',
        end_date='',
        before_start_days=90
    )

    m6 = M.derived_feature_extractor.v3(
        input_data=m2.data,
        features=m3.data,
        date_col='date',
        instrument_col='instrument',
        drop_na=False,
        remove_extra_columns=False,
        user_functions={}
    )

    m5 = M.cached.v3(
        input_1=m6.data,
        run=m5_run_bigquant_run,
        post_run=m5_post_run_bigquant_run,
        input_ports='',
        params='{}',
        output_ports=''
    )

    df=m5.data_1.read()    
    df=df[df.index>=start_date]
    return df    
####
def bar_tras(bar,tras):
    '''
    bar：日线数据
    tras：交易记录
    链接两个数据，并且设置sell，buy列
    
    '''
    #合并两个数据
    bar.reset_index(inplace=True)
    bar=bar.merge(tras,on='date',how='left')

    #添加两列
    bar['sell']=np.nan
    bar['buy']=np.nan
    #设置买入卖出点
    def buy_or_sell(x):
        if x.transaction_money>0:
            x.buy=x.low
        if x.transaction_money<0:
            x.sell=x.high
        return x
    #应用
    bar=bar.apply(lambda x: buy_or_sell(x),axis=1)
    #设置日期
    bar.set_index('date',drop=True,inplace=True)
    return bar

top,bottom,tras=transaction_money(m19)

# 循环画图
for sid in bottom.index:
    
    sel_tras=tras[tras.sid==sid]    
    sel_tras['date']=sel_tras.dt.apply(lambda x :pd.Timestamp(x.strftime('%Y-%m-%d')))
    #设置日期
    start_date= ( sel_tras.date.iloc[0]- datetime.timedelta(days=50)).strftime('%Y-%m-%d') 
    end_date= (sel_tras.date.iloc[-1] + datetime.timedelta(days=50)).strftime('%Y-%m-%d') 

    signal='''ta_atr_28_0/close_0*100
    '''
    #读取数据
    bar=get_bars(sid,start_date,end_date,signal)
    df=bar_tras(bar,sel_tras)
    #选择画图数据
    select=['open', 'high', 'low', 'close','ta_sma_30_0','sell','buy']    
    #把 signal 加入到select 
    for i in signal.strip().split('\n'):
        select.append(i.strip())
        
    print(sid,'-'*40)
    T.plot(df[select],  options={'chart': {'title': sid, 'height': 800}, 'series': [{},{'type': 'line','yAxis':0},{'type': 'scatter','yAxis':0,'color':'green'},{'type': 'scatter','yAxis':0,'color':'red'}]},stock=True, candlestick=True)
    print('--------------\n'*2)
    

[2019-04-15 17:05:19.085617] INFO: bigquant: instruments.v2 开始运行..

[2019-04-15 17:05:19.098958] INFO: bigquant: instruments.v2 运行完成[0.013309s].

[2019-04-15 17:05:19.103250] INFO: bigquant: input_features.v1 开始运行..

[2019-04-15 17:05:19.115241] INFO: bigquant: input_features.v1 运行完成[0.011984s].

[2019-04-15 17:05:19.124721] INFO: bigquant: general_feature_extractor.v7 开始运行..

...

[2019-04-15 17:05:26.063690] INFO: bigquant: derived_feature_extractor.v3 运行完成[0.076703s].

[2019-04-15 17:05:26.069145] INFO: bigquant: cached.v3 开始运行..

[2019-04-15 17:05:26.119630] INFO: bigquant: cached.v3 运行完成[0.050479s].

300028.SZA ----------------------------------------


--------------
--------------

